# リョウコと実装！量子機械学習で手書き文字認識

みなさん、「リョウコと実装！量子機械学習で手書き文字認識」へようこそ。機械学習は、量子コンピューターの応用範囲として期待されているエリアのひとつです。本ハンズオンでは、機械学習の中でも量子古典ハイブリッド・ニューラルネットワークを用いた手書き文字認識を取り上げます。MNISTを利用した量子古典ハイブリッド・モデルの学習方法と使用方法を紹介し、実際に手書き文字を認識するWebアプリを体感していただきます！

前提知識：Python 、ニューラルネットワーク、IBM Cloud

事前準備：[IBM Quantum](https://quantum-computing.ibm.com/)へのサインアップ

## 目次

1. [はじめに](#introduction)
1. [量子古典ハイブリッド・ニューラルネットワーク](#hybrid)
1. [実装！](#implementation)
1. [まとめ](#summary)
1. [参考文献](#reference)

## はじめに <a id='indroduction'></a>
まずは、ハンズオンを実行する環境を準備しましょう。

1. ハンズオンで使用するJupyter notebookファイル(zipファイル)を[こちら](https://github.com/purepureclub/RyokoHQNN/archive/refs/heads/main.zip)からダウンロードします
1. ダウンロードしたzipファイルを解凍します
1. [IBM Quantum](https://quantum-computing.ibm.com/) にログインします
1. IBM QuantumのDashbord上のLaunch Labボタンをクリックします
1. Upload Filesボタン(上矢印)を押して、解凍したファイルをアップロードします
1. アップロードしたファイル「Handson.ipynb」を開いてください

次に、必要なモジュールとライブラリをインポートしておきましょう。Jupyter notebookでは、セルにカーソルを置き、Shift+Enterを押すと、セル内のコードが実行されます。

In [ ]:
# Necessary imports
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import cat, no_grad, manual_seed
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim
from torch.nn import (
    Module,
    Conv2d,
    Linear,
    Dropout2d,
    NLLLoss,
)
import torch.nn.functional as F

from qiskit import QuantumCircuit
from qiskit.utils import algorithm_globals
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

# Set seed for random generators
algorithm_globals.random_seed = 42

以上で準備は完了です。

## 量子古典ハイブリッド・ニューラルネットワーク <a id='hybrid'></a>

ニューラルネットワークは、ヒトのニューロンを模したシステムで、一般的に入力層に入れられたデータをもとに、出力層の数だけ分類します。
学習により、重みやバイアスを決めて、モデルを定義します。
![ニューラルネットワーク](NeuralNetwork.png)

量子古典ハイブリッド・ニューラルネットワークとは、その名の通り量子コンピューターと古典コンピューターのハイブリッドなニューラルネットワークのことです。古典ニューラルネットワークの一部の層を、量子コンピューターで計算する量子層で置き換えます。Qiskitの`TorchConnector`クラスを使用すると、`PyTorch`の ワークフローにQiskitで定義した量子層(`NeuralNetwork`クラス)を統合することができます。このようにして作成されたネットワークは、PyTorchの古典アーキテクチャーにシームレスに組み込むことができます。追加の考慮事項なしに古典層・量子層を同時に学習・テストすることができるのです。
![ハイブリッド・ニューラルネットワーク](HybridNeuralNetwork1.png)

量子層は一般的に二つのパラメーター付き量子回路で実装されます。一つは古典層からの出力($\vec{x}$)を量子データに変換(エンコーディング)する特徴マップ、もう一つは量子層の出力を計算するAnsatzです。Ansatzには最適化により決定されるパラメーター($\vec{\theta}$)が使用されます。Qiskitは特徴マップ、Ansatzともに簡単に実装できるクラスを提供していますので、その使い方を見ていきましょう。

![Quantum Neural Network](QuantumNeuralNetwork1.png)


## 実装！<a id='implementation'></a>

1. ステップ 1: データの準備
1. ステップ 2: 量子古典ハイブリッド・ニューラルネットワークの構築
1. ステップ 3: 学習
1. ステップ 4: テスト


### ステップ 1: データの準備
まず、torchvision API を利用して、 MNIST データセットをダウンロード(約70MB)し、学習用のMNISTデータ100件をロードします。

In [ ]:
# Train Dataset
# -------------

# Set train shuffle seed (for reproducibility)
manual_seed(42)

batch_size = 1
n_samples = 100  # We will concentrate on the first 100 samples

# Use pre-defined torchvision function to load MNIST train data
X_train = datasets.MNIST(
    root="./data", train=True, download=True, transform=transforms.Compose([transforms.ToTensor()])
)

# Filter out labels (originally 0-9), leaving only labels 0 and 1
idx = np.append(
    np.where(X_train.targets == 0)[0][:n_samples], np.where(X_train.targets == 1)[0][:n_samples]
)
X_train.data = X_train.data[idx]
X_train.targets = X_train.targets[idx]

# Define torch dataloader with filtered data
train_loader = DataLoader(X_train, batch_size=batch_size, shuffle=True)

サンプルデータを6件表示させてみましょう。手書きの0と1の画像で構成されていることが確認できます。

In [ ]:
n_samples_show = 6

data_iter = iter(train_loader)
fig, axes = plt.subplots(nrows=1, ncols=n_samples_show, figsize=(10, 3))

while n_samples_show > 0:
    images, targets = data_iter.__next__()

    axes[n_samples_show - 1].imshow(images[0, 0].numpy().squeeze(), cmap="gray")
    axes[n_samples_show - 1].set_xticks([])
    axes[n_samples_show - 1].set_yticks([])
    axes[n_samples_show - 1].set_title("Labeled: {}".format(targets[0].item()))

    n_samples_show -= 1

同様にして、テスト用データ50件をロードします。

In [ ]:
# Test Dataset
# -------------

# Set test shuffle seed (for reproducibility)
# manual_seed(5)

n_samples = 50

# Use pre-defined torchvision function to load MNIST test data
X_test = datasets.MNIST(
    root="./data", train=False, download=True, transform=transforms.Compose([transforms.ToTensor()])
)

# Filter out labels (originally 0-9), leaving only labels 0 and 1
idx = np.append(
    np.where(X_test.targets == 0)[0][:n_samples], np.where(X_test.targets == 1)[0][:n_samples]
)
X_test.data = X_test.data[idx]
X_test.targets = X_test.targets[idx]

# Define torch dataloader with filtered data
test_loader = DataLoader(X_test, batch_size=batch_size, shuffle=True)

### ステップ2：量子古典ハイブリッド・ニューラルネットワークの構築
量子古典ハイブリッド・ニューラルネットワークを構築するために、まず量子層を定義しましょう。今回は、Qiskitの[`EstimatorQNN`](https://qiskit.org/ecosystem/machine-learning/locale/ja_JP/stubs/qiskit_machine_learning.neural_networks.EstimatorQNN.html)クラスを使用します。`EstimatorQNN`クラスは、特徴マップとAnsatzをパラメーター化された量子回路を入力として取り込み、フォワード・パスやバックワード・パスの期待値を計算し出力します。今回は特徴マップに`ZZFeatureMap`を、Ansatzに`RealAmplitude`を使用します。後述しますが、今回量子層への入力(古典層からの出力)は2つですので、2量子ビットの`ZZFeatureMap`と`RealAmplitude`を作成します。

![量子層のイメージ](QuantumNeuralNetwork2.png)

[`ZZFeatureMap`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.ZZFeatureMap.html) : Pauli行列を用いたエンコーディング方法([`PauliFeatureMap`](https://qiskit.org/documentation/stable/0.24/stubs/qiskit.circuit.library.PauliFeatureMap.html))の一つで、Paul行列のZとZZのみを用いたエンコーディング

[`RealAmplitudes`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.RealAmplitudes.html) : 化学アプリケーションや機械学習における分類回路のAnsatzとして用いられるヒューリスティックな関数。回転と量子もつれの交互の層で構成。結果として得られる量子状態は実振幅のみを持ち、複素数部分は常に0であるため、`RealAmplitudes`と呼ばれる

In [ ]:
# Define and create QNN
def create_qnn():
    feature_map = ZZFeatureMap(2)
    ansatz = RealAmplitudes(2, reps=1)
    qc = QuantumCircuit(2)
    qc.compose(feature_map, inplace=True)
    qc.compose(ansatz, inplace=True)

    # REMEMBER TO SET input_gradients=True FOR ENABLING HYBRID GRADIENT BACKPROP
    qnn = EstimatorQNN(
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        input_gradients=True,
    )
    return qnn

qnn1 = create_qnn()
qnn1.circuit.draw()

実際のどのようなゲートで実装されているか確認しましょう。

In [ ]:
qnn1.circuit.decompose().draw()

量子層を含めた量子・古典ハイブリッドニューラルネットワークを構築します。今回作成するネットワークの概念図は下図のようになります。
![ハイブリッド・ニューラルネットワーク](HybridNeuralNetwork2.png)

In [ ]:
# Define torch NN module
class Net(Module):
    def __init__(self, qnn):
        super().__init__()
        self.conv1 = Conv2d(1, 2, kernel_size=5)
        self.conv2 = Conv2d(2, 16, kernel_size=5)
        self.dropout = Dropout2d()
        self.fc1 = Linear(256, 64)
        self.fc2 = Linear(64, 2)  # 2-dimensional input to QNN
        self.qnn = TorchConnector(qnn)  # Apply torch connector, weights chosen
        # uniformly at random from interval [-1,1].
        self.fc3 = Linear(1, 1)  # 1-dimensional output from QNN

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.qnn(x)  # apply QNN
        x = self.fc3(x)
        return cat((x, 1 - x), -1)

model1 = Net(qnn1)

### ステップ3: 学習！
最適化アルゴリズムと損失関数を定義して、学習を始めます。今回は最適化アルゴリズムにAdam、損失関数にNLLossを使用します。Adamについては[こちら](https://qiita.com/omiita/items/1735c1d048fe5f611f80)を、NLLossについては[こちら](https://qiita.com/y629/items/1369ab6e56b93d39e043)が参考になると思います。

In [ ]:
# Define model, optimizer, and loss function
optimizer = optim.Adam(model1.parameters(), lr=0.001)
loss_func = NLLLoss()

# Start training
epochs = 10  # Set number of epochs
loss_list = []  # Store loss history
model1.train()  # Set model to training mode

for epoch in range(epochs):
    total_loss = []
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad(set_to_none=True)  # Initialize gradient
        output = model1(data)  # Forward pass
        loss = loss_func(output, target)  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize weights
        total_loss.append(loss.item())  # Store loss
    loss_list.append(sum(total_loss) / len(total_loss))
    print("Training [{:.0f}%]\tLoss: {:.4f}".format(100.0 * (epoch + 1) / epochs, loss_list[-1]))

学習には数分かかりますので、その間にIBM Cloudでの呼び出し方法を見てみましょう。
学習を終えると、`torch.save` メソッドでモデルを保存できます。このモデルを利用したCloud上のアプリで、手書き文字を認識してみましょう。

![Sample Application on IBM Cloud](Cloud1.png)

![Architecture on IBM Cloud](Cloud2.png)

学習が終わったら、学習の様子をプロットしてみましょう。

In [ ]:
# Plot loss convergence
plt.plot(loss_list)
plt.title("Hybrid NN Training Convergence")
plt.xlabel("Training Iterations")
plt.ylabel("Neg. Log Likelihood Loss")
plt.show()

モデルを保存して学習は終了です。

In [ ]:
torch.save(model1.state_dict(), "model1.pt")

### ステップ4: テスト 
保存したモデルを、実行用にロードします。

In [ ]:
# Create a new instance of Hybrid QNN and load trained model
qnn2 = create_qnn()
model2 = Net(qnn2)
model2.load_state_dict(torch.load("model1.pt"))

テストデータを分類してみましょう。

In [ ]:
model2.eval()  # set model to evaluation mode
with no_grad():

    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        output = model2(data)
        if len(output.shape) == 1:
            output = output.reshape(1, *output.shape)

        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

        loss = loss_func(output, target)
        total_loss.append(loss.item())

    print(
        "Performance on test data:\n\tLoss: {:.4f}\n\tAccuracy: {:.1f}%".format(
            sum(total_loss) / len(total_loss), correct / len(test_loader) / batch_size * 100
        )
    )

非常に精度のよいモデルができたようですね。実際の予測結果を6件表示させてみましょう。

In [ ]:
# Plot predicted labels

n_samples_show = 6
count = 0
fig, axes = plt.subplots(nrows=1, ncols=n_samples_show, figsize=(10, 3))

model2.eval()
with no_grad():
    for batch_idx, (data, target) in enumerate(test_loader):
        if count == n_samples_show:
            break
        output = model2(data[0:1])
        if len(output.shape) == 1:
            output = output.reshape(1, *output.shape)

        pred = output.argmax(dim=1, keepdim=True)

        axes[count].imshow(data[0].numpy().squeeze(), cmap="gray")

        axes[count].set_xticks([])
        axes[count].set_yticks([])
        axes[count].set_title("Predicted {}".format(pred.item()))

        count += 1

## まとめ
今回は量子古典ハイブリッド・ニューラルネットワークを利用した、手書き文字認識モデルを作成し、IBM Cloud上のWebアプリから使用してみました。
Qiskitの`TorchConnector`クラスを使うと、PytorchのワークフローにQiskitの`NewralNetwork`クラスを簡単に組み込むことができます。
学習データに応じて、適切な特徴マップ、Ansatzを利用すれば、量子優位性を持ったモデルを作成することができるかもしれません。

## 参考文献 <a id='reference'></a>
[1] [Torch コネクターおよびハイブリッド QNN](https://qiskit.org/documentation/machine-learning/locale/ja_JP/tutorials/05_torch_connector.html)

[2] [PyTorchとQiskitを用いた量子古典ハイブリッド・ニューラル・ネットワーク](https://qiskit.org/textbook/ja/ch-machine-learning/machine-learning-qiskit-pytorch.html)

In [1]:
import qiskit.tools.jupyter

%qiskit_version_table
%qiskit_copyright

ワークショップ、セッション、および資料は、IBMまたはセッション発表者によって準備され、それぞれ独自の見解を反映したものです。それらは情報提供の目的のみで提供されており、いかなる参加者に対しても法律的またはその他の指導や助言を意図したものではなく、またそのような結果を生むものでもありません。本講演資料に含まれている情報については、完全性と正確性を期するよう努力しましたが、「現状のまま」提供され、明示または暗示にかかわらずいかなる保証も伴わないものとします。本講演資料またはその他の資料の使用によって、あるいはその他の関連によって、いかなる損害が生じた場合も、IBMは責任を負わないものとします。本講演資料に含まれている内容は、IBMまたはそのサプライヤーやライセンス交付者からいかなる保証または表明を引きだすことを意図したものでも、IBMソフトウェアの使用を規定する適用ライセンス契約の条項を変更することを意図したものでもなく、またそのような結果を生むものでもありません。

本講演資料でIBM製品、プログラム、またはサービスに言及していても、IBMが営業活動を行っているすべての国でそれらが使用可能であることを暗示するものではありません。本講演資料で言及している製品リリース日付や製品機能は、市場機会またはその他の要因に基づいてIBM独自の決定権をもっていつでも変更できるものとし、いかなる方法においても将来の製品または機能が使用可能になると確約することを意図したものではありません。本講演資料に含まれている内容は、参加者が開始する活動によって特定の販売、売上高の向上、またはその他の結果が生じると述べる、または暗示することを意図したものでも、またそのような結果を生むものでもありません。パフォーマンスは、管理された環境において標準的なIBMベンチマークを使用した測定と予測に基づいています。ユーザーが経験する実際のスループットやパフォーマンスは、ユーザーのジョブ・ストリームにおけるマルチプログラミングの量、入出力構成、ストレージ構成、および処理されるワークロードなどの考慮事項を含む、数多くの要因に応じて変化します。したがって、個々のユーザーがここで述べられているものと同様の結果を得られると確約するものではありません。

記述されているすべてのお客様事例は、それらのお客様がどのようにIBM製品を使用したか、またそれらのお客様が達成した結果の実例として示されたものです。実際の環境コストおよびパフォーマンス特性は、お客様ごとに異なる場合があります。

IBM、IBM ロゴ、ibm.com、IBM Cloud、Qiskitは、世界の多くの国で登録されたInternational  Business  Machines  Corporationの商標です。他の製品名およびサービス名等は、それぞれIBMまたは各社の商標である場合があります。現時点でのIBM の商標リストについては、www.ibm.com/legal/copytrade.shtml をご覧ください。

Red HatおよびOpenShift は、米国およびその他の国における Red Hat, Inc. またはその子会社の登録商標です。

JupyterはNumFOCUS foundationの登録商標です。

PythonはPython Software Foundationの登録商標です。

その他のすべての商標は、それぞれの所有者に帰属します。